In [1]:
import requests
import pandas as pd

file_url = 'https://raw.githubusercontent.com/ani-exe/simple_ao3_scraper/main/outfile.json'
data = requests.get(file_url).json()
data = data['data']
df = pd.DataFrame(data)

In [2]:
# kudos / hits can be useful for another view (altho favors newer works & oneshots)
# versus overall stats which favors multichap & older
# wonder what the balance would be here.

df = df.assign(kudos_per_hits = df.kudos / df.hits)
df['lastUpdated'] = pd.to_datetime(df['lastUpdated'])
df

,title,author,isAnon,isOrphan,id,rating,warnings,category,isWip,lastUpdated,...,language,words,currentChapters,intendedChapters,kudos,hits,comments,bookmarks,collections,kudos_per_hits
0,Trust,mikanviola,False,False,/works/47233843,Teen And Up Audiences,[No Archive Warnings Apply],[M/M],False,2023-05-17,...,English,2863,2,2,4,14,1,0,0,0.285714
1,At The End of Everything (Hold Onto Anything),FixhBones (Fish_Bones),False,False,/works/47236459,Teen And Up Audiences,[No Archive Warnings Apply],"[Gen, M/M]",False,2023-05-17,...,English,1771,1,1,9,37,0,1,0,0.243243
2,Warmth Is in the Empty Spaces of the Heart,calfhooves,False,False,/works/47235577,Teen And Up Audiences,[No Archive Warnings Apply],[M/M],False,2023-05-17,...,English,2412,1,1,9,42,3,1,0,0.214286
3,Tabata,metukika,False,False,/works/46992460,General Audiences,[No Archive Warnings Apply],[M/M],False,2023-05-17,...,English,13774,3,3,83,708,38,10,0,0.117232
4,New Business Venture,blue_avocado,False,False,/works/18739756,Teen And Up Audiences,[No Archive Warnings Apply],[Gen],True,2023-05-17,...,English,44304,9,?,23,320,10,2,0,0.071875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7059,Double Team Tactic,orphan_account,False,True,/works/5782909,Mature,[Underage],[M/M],False,2016-01-22,...,English,5281,1,1,792,12710,41,62,0,0.062313
7060,1,slutreigen (pleasejustno),False,False,/works/5613691,General Audiences,[No Archive Warnings Apply],[No category],False,2016-01-02,...,English,356,1,1,204,2221,15,5,0,0.091851
7061,Brother's Keeper,allthingsunrelated,False,False,/works/5239292,Not Rated,[Choose Not To Use Archive Warnings],[Gen],False,2015-11-18,...,English,1722,1,1,251,1931,7,43,0,0.129984
7062,senses,ekubo (apurochi),False,False,/works/2535437,Teen And Up Audiences,[Underage],[M/M],False,2014-10-30,...,English,266,1,1,172,2786,5,5,1,0.061737


In [3]:
df.describe()

,words,currentChapters,kudos,hits,comments,bookmarks,collections,kudos_per_hits
count,7.064000e+03,7064.000000,7064.000000,7064.000000,7064.000000,7064.000000,7064.000000,7064.000000
mean,6.590518e+03,2.178228,207.103907,2182.405011,22.996319,23.256370,0.071631,0.110478
std,6.790334e+04,13.022827,395.214921,6097.343447,102.285764,70.852027,0.299548,0.050869
min,1.000000e+02,1.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000
25%,1.148500e+03,1.000000,47.000000,487.000000,4.000000,3.000000,0.000000,0.071931
50%,2.269000e+03,1.000000,109.000000,1065.000000,8.000000,8.000000,0.000000,0.105413
75%,4.940500e+03,1.000000,225.250000,2236.000000,18.000000,21.000000,0.000000,0.142618
max,5.440995e+06,1056.000000,12500.000000,386291.000000,5886.000000,2703.000000,8.000000,0.475000


In [4]:
numeric_stats = ["kudos", "hits", "comments", "bookmarks", "kudos_per_hits"]
df[["rating"] + numeric_stats].groupby("rating").mean()

,kudos,hits,comments,bookmarks,kudos_per_hits
rating,,,,,
Explicit,208.994012,3477.519960,17.473054,18.754491,0.061050
General Audiences,206.848496,1632.233780,15.143530,22.006887,0.134563
Mature,182.157706,2335.211470,38.611111,20.508961,0.084211
Not Rated,125.886555,1190.884454,10.897059,11.903361,0.107240
Teen And Up Audiences,229.752755,2449.886294,33.682239,29.821067,0.110157


In [5]:
split_rel_df = df.relationships.apply(pd.Series).add_prefix('relationship_')
split_rel_df.describe()

,relationship_0,relationship_1,relationship_2,relationship_3,relationship_4,relationship_5,relationship_6,relationship_7,relationship_8,relationship_9,relationship_10,relationship_11,relationship_12,relationship_13,relationship_14,relationship_15
count,6172,2073,965,546,287,155,87,55,33,22,13,7,5,3,2,2
unique,389,347,229,176,114,74,51,39,26,16,12,7,5,2,2,2
top,Reigen Arataka/Serizawa Katsuya,"Kageyama ""Mob"" Shigeo & Reigen Arataka","Kageyama ""Mob"" Shigeo & Reigen Arataka","Kageyama ""Mob"" Shigeo & Reigen Arataka","Kageyama ""Mob"" Shigeo & Reigen Arataka","Kageyama Ritsu & Kageyama ""Mob"" Shigeo",Reigen Arataka/Serizawa Katsuya,"Hanazawa Teruki/Kageyama ""Mob"" Shigeo",Reigen Arataka & Serizawa Katsuya,Kurata Tome/Takane Tsubomi,"Kageyama ""Mob"" Shigeo & Kurata Tome","Kageyama ""Mob"" Shigeo & Kurata Tome","Hanazawa Teruki & Kageyama ""Mob"" Shigeo",Dimple & Reigen Arataka,Emi/Mezato Ichi,"Emi & Kageyama ""Mob"" Shigeo"
freq,1145,209,77,31,17,12,7,7,4,3,2,1,1,2,1,1


In [6]:
# i still need to mess around with ^
